In [1]:
import pandas as pd
import os
import numpy as np
from copy import deepcopy

In [2]:
st = pd.HDFStore(os.path.expanduser("~/all.h5"))

In [3]:
metadata_df = pd.read_csv("metadata/metadata.csv",index_col=0)

In [4]:
feeds = {'use':'aggregate',
        'air1':'hvac',
         'clotheswasher1':'wm',
         'dishwasher1':'dw',
         'microwave1':'mw',
         'refrigerator1':'fridge',
         'oven1':'oven',
        'car1':'ec',
         'waterheater1':'wh',
         'drye1':'dr'
        }

In [5]:
cities = ['Austin','Houston','San Diego','Boulder']

In [31]:
years = {
    2014:{'start':'1-1-2014', 'end':'12-31-2014', 'start_month':1, 'end_month':12, 'months':1},
    2015:{'start':'1-1-2015', 'end':'12-31-2015', 'start_month':1, 'end_month':12, 'months':1},
    2016:{'start':'1-1-2016', 'end':'12-31-2016', 'start_month':1, 'end_month':12, 'months':1}
    }

In [33]:
start, end

('1-1-2015', '12-31-2015')

In [34]:
out_overall = {}
for year, year_dict in years.iteritems(): 
    out_overall[year] = {}
    to_ignore = []
    for city in cities[:]:
        print city, year
        city_data = metadata_df[metadata_df['city'] == city]
        city_homes = city_data.index.values.astype('int')
        out = {}
        for home in city_homes[:]:
            if home in to_ignore:
                continue
            try:
                data_df = st['/%d' %home][year_dict['start']:year_dict['end']]
                data_df_resampled = data_df.resample("1M", how="sum")
                df_res_kwh = data_df_resampled.div(1000)
                df_res_kwh.index = df_res_kwh.index.month
                temp = {}
                for feed_name, feed_value in feeds.iteritems():
                    temp[feed_value] = []
                    for month in range(year_dict['start_month'], year_dict['end_month']+1):


                        try:
                            temp[feed_value].append(df_res_kwh.loc[month, feed_name])
                        except:
                            temp[feed_value].append(np.NAN)
                out[home] = np.hstack(temp.values())




            except Exception, e:
                pass
        appliances = temp.keys()
        col_headers = []
        for appliance in appliances:
            for month in range(start_month, end_month+1):
                col_headers.append(appliance+"_"+str(month))
        df = pd.DataFrame(out).T
        df.columns = col_headers

        # Read metadata
        mdf = pd.read_csv("metadata/%s_md.csv" %city.replace(" ",""), index_col=0)
        for column in mdf.columns:
            df[column] = mdf[column]

        city = city.replace(" ","")
        out_overall[year][city] =df

        # Set metadata available flag
        #df['md_available'] = (df['area']>0).astype('int')

        tdf = df[['aggregate_'+str(month) for month in range(start_month, end_month+1)]].dropna().copy()
        full_agg = {}
        min_max_ratio = {}
        difference_min_max_ratio = {}
        variance = {}
        kurt = {}
        skew= {}
        p_25 = {}
        p_75 = {}
        p_50 = {}

        for home in df.index:
            if home in tdf.index:
                full_agg[home] = 1
                difference_min_max_ratio[home] = (tdf.ix[home].max()-tdf.ix[home].min())/tdf.ix[home].max()
                min_max_ratio[home] = tdf.ix[home].min()/tdf.ix[home].max()
                variance[home] = tdf.ix[home].var()
                p_25[home] =np.percentile(tdf.ix[home], 25)
                p_50[home] =np.percentile(tdf.ix[home], 50)
                p_75[home] = np.percentile(tdf.ix[home], 75)
                kurt[home] = tdf.ix[home].kurtosis()
                skew[home] = tdf.ix[home].skew()

            else:
                full_agg[home] = 0
                min_max_ratio[home] = np.NaN
                difference_min_max_ratio[home] = np.NaN
                p_25[home] = np.NaN
                p_50[home] = np.NaN
                p_75[home] = np.NaN
                skew[home] = np.NaN
                kurt[home] = np.NaN
                variance[home] = np.NaN


        df['full_agg_available'] = pd.Series(full_agg)
        df['ratio_min_max'] = pd.Series(min_max_ratio)
        df['difference_ratio_min_max'] = pd.Series(min_max_ratio)
        df['variance'] = pd.Series(variance)
        df['skew'] = pd.Series(skew)
        df['kurtosis'] = pd.Series(kurt)
        df['p_25'] = pd.Series(p_25)
        df['p_50'] = pd.Series(p_50)
        df['p_75'] = pd.Series(p_75)

Austin 2016


/home/nipun/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()


Houston 2016
San Diego 2016
Boulder 2016
Austin 2014
Houston 2014
San Diego 2014
Boulder 2014
Austin 2015
Houston 2015
San Diego 2015
Boulder 2015


In [12]:
import pickle

In [40]:
out_overall[2014]['Austin'].head()

,hvac_1,hvac_2,hvac_3,hvac_4,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10,...,num_occupants,full_agg_available,ratio_min_max,difference_ratio_min_max,variance,skew,kurtosis,p_25,p_50,p_75
22,138.796936,116.709915,51.263603,63.073967,96.288300,185.116989,223.729828,262.991455,150.238037,75.410019,...,NaN,1,0.549180,0.549180,3823.597452,0.516786,-0.689573,274.504639,315.106934,369.923645
26,7.635717,2.044650,49.361549,81.417267,236.560089,493.189484,387.065552,841.948120,504.574371,335.706543,...,4.0,1,0.388645,0.388645,100640.896298,1.163045,1.135843,748.288940,898.211029,1112.968536
48,26.711552,70.193047,28.318184,81.183067,109.748268,166.510284,185.771530,205.711853,172.368317,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,724.853577,508.429138,236.376205,44.645180,31.479816,262.018738,298.577332,540.575989,275.283661,84.855850,...,7.0,1,0.452525,0.452525,127069.291845,1.049623,0.234731,1056.249817,1111.055847,1487.265900
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,0.351272,0.351272,3673.068633,-1.190632,2.583459,262.236763,287.146225,303.242020


In [41]:
pickle.dump(out_overall, open('metadata/all_regions_years.pkl','w'))